In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [2]:
dataset_path = "/kaggle/input/euro-usd1h/cleaned_data_EUROUSD_1H.csv"
dataset_raw = pd.read_csv(dataset_path)
dataset_raw = dataset_raw.dropna(subset=['Close'])
dataset_raw['future_close'] = dataset_raw['Close'].shift(-1)
dataset_raw = dataset_raw.dropna(subset=['future_close'])
dataset_raw['label'] = (dataset_raw['future_close'] > dataset_raw['Close']).astype(int)
#dataset['label'] = ((dataset['future_close'] / dataset['Close'] - 1) > 0.002).astype(int)

dataset = feature_extraction(dataset_raw)
dataset.dropna(axis=0, how='any', inplace=True)

Y=dataset.iloc[:,9]
#X = dataset.drop(columns=['label', 'future_close'])
X=dataset.iloc[:,10:]
print(Y)
#Y = dataset['label']

# Scale features

        Date      Time     Open     High      Low    Close  Volume  Spread
0  9/21/2009  11:00:00  1.46594  1.46709  1.46532  1.46653    3595       8
1  9/21/2009  12:00:00  1.46644  1.46690  1.46117  1.46264    3604       8
2  9/21/2009  13:00:00  1.46267  1.46601  1.46082  1.46475    3694       7
3  9/21/2009  14:00:00  1.46470  1.46626  1.46344  1.46554    4180       9
4  9/21/2009  15:00:00  1.46549  1.47007  1.46460  1.46883    4284       9


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.98)
principal_components = pca.fit_transform(X.dropna())

            Date      Time     Open     High      Low    Close  Volume  \
0      9/21/2009  11:00:00  1.46594  1.46709  1.46532  1.46653    3595   
1      9/21/2009  12:00:00  1.46644  1.46690  1.46117  1.46264    3604   
2      9/21/2009  13:00:00  1.46267  1.46601  1.46082  1.46475    3694   
3      9/21/2009  14:00:00  1.46470  1.46626  1.46344  1.46554    4180   
4      9/21/2009  15:00:00  1.46549  1.47007  1.46460  1.46883    4284   
...          ...       ...      ...      ...      ...      ...     ...   
99994  10/1/2025   2:00:00  1.17393  1.17395  1.17322  1.17378    1396   
99995  10/1/2025   3:00:00  1.17378  1.17390  1.17338  1.17378    1323   
99996  10/1/2025   4:00:00  1.17381  1.17585  1.17379  1.17579    1981   
99997  10/1/2025   5:00:00  1.17578  1.17669  1.17520  1.17521    3803   
99998  10/1/2025   6:00:00  1.17523  1.17748  1.17505  1.17678    5548   

       Spread  future_close  label  
0           8       1.46264      0  
1           8       1.46475      1  


In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(principal_components)

,Date,Time,Open,High,Low,Close,Volume,Spread,future_close,label,ret1,ma_5,ma_20,ma_diff,vol_20,ema_fast,ema_slow,ema_cross
20,9/22/2009,7:00:00,1.47370,1.47907,1.47206,1.47786,4344,8,1.47834,1,0.002734,1.473132,1.469073,0.004059,0.001161,1.471517,1.469542,0.001975
21,9/22/2009,8:00:00,1.47775,1.47997,1.47697,1.47834,4616,9,1.48046,1,0.000325,1.474494,1.469858,0.004636,0.000917,1.472566,1.470194,0.002373
22,9/22/2009,9:00:00,1.47828,1.48119,1.47761,1.48046,5264,9,1.47940,0,0.001434,1.476334,1.470643,0.005691,0.000916,1.473781,1.470954,0.002827
23,9/22/2009,10:00:00,1.48054,1.48204,1.47880,1.47940,5021,7,1.47822,0,-0.000716,1.477978,1.471337,0.006641,0.000958,1.474645,1.471580,0.003066
24,9/22/2009,11:00:00,1.47939,1.47989,1.47786,1.47822,3547,8,1.47938,1,-0.000798,1.478856,1.471806,0.007050,0.000901,1.475195,1.472072,0.003124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,10/1/2025,2:00:00,1.17393,1.17395,1.17322,1.17378,1396,5,1.17378,0,-0.000094,1.173616,1.174095,-0.000479,0.000805,1.173787,1.173622,0.000165
99995,10/1/2025,3:00:00,1.17378,1.17390,1.17338,1.17378,1323,3,1.17579,1,0.000000,1.173692,1.174076,-0.000384,0.000800,1.173786,1.173634,0.000152
99996,10/1/2025,4:00:00,1.17381,1.17585,1.17379,1.17579,1981,3,1.17521,0,0.001712,1.174162,1.174089,0.000073,0.000849,1.174094,1.173794,0.000300
99997,10/1/2025,5:00:00,1.17578,1.17669,1.17520,1.17521,3803,5,1.17678,1,-0.000493,1.174490,1.174109,0.000381,0.000843,1.174266,1.173899,0.000367


In [ ]:
def create_sequences(X, y, window=30):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, Y, window=10)

,ret1,ma_5,ma_20,ma_diff,vol_20,ema_fast,ema_slow,ema_cross
20,0.002734,1.473132,1.469073,0.004059,0.001161,1.471517,1.469542,0.001975
21,0.000325,1.474494,1.469858,0.004636,0.000917,1.472566,1.470194,0.002373
22,0.001434,1.476334,1.470643,0.005691,0.000916,1.473781,1.470954,0.002827
23,-0.000716,1.477978,1.471337,0.006641,0.000958,1.474645,1.471580,0.003066
24,-0.000798,1.478856,1.471806,0.007050,0.000901,1.475195,1.472072,0.003124
...,...,...,...,...,...,...,...,...
1215,-0.000140,1.501768,1.501530,0.000237,0.001080,1.501296,1.501079,0.000216
1216,-0.000546,1.501186,1.501363,-0.000177,0.001036,1.501176,1.501038,0.000138
1217,-0.000400,1.500982,1.501164,-0.000182,0.001037,1.500983,1.500955,0.000028
1218,0.001107,1.500982,1.500979,0.000003,0.001048,1.501075,1.501001,0.000073


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
fold = 1
accuracy_scores = []

for train_index, val_index in tscv.split(X_seq):
    X_train, X_val = X_seq[train_index], X_seq[val_index]
    y_train, y_val = y_seq[train_index], y_seq[val_index]

    # Step 5: Build CNN-LSTM model
    model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
    ])
    callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(patience=5, factor=0.5)]

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history=model.fit(X_train, y_train, validation_data=(X_val, y_val), 
    epochs=10, batch_size=64, callbacks=callbacks)
    
    # Step 7: Evaluate
    y_pred_prob = model.predict(X_val)
    
    # Convert probabilities to class labels (0 or 1)
    y_pred = (y_pred_prob > 0.5).astype(int).squeeze()
    
    # Calculate accuracy
    accuracy = np.mean(y_pred == y_val)
    accuracy_scores.append(accuracy)
    
    print(f"Fold {fold} Accuracy: {accuracy:.4f}")
    fold += 1

print(f"Average Accuracy across folds: {np.mean(accuracy_scores):.4f}")

array([[ 0.30658557, -0.96878314, -0.88207143, ..., -0.95016803,
        -0.87870023, -0.72395204],
       [ 0.61763406,  0.75782619,  0.65082124, ...,  0.71830377,
         0.60373698,  1.01759337],
       [-1.06795833, -1.1403987 , -1.2961346 , ..., -1.19701226,
        -1.1428395 , -0.63343204],
       ...,
       [ 0.79116941,  1.74120368,  1.94436141, ...,  1.85778057,
         1.81172757,  0.68770259],
       [ 0.18371651,  1.09220067,  1.05714665, ...,  1.07013269,
         1.07630782,  0.14207912],
       [-1.49700819,  1.88044168,  1.9224439 , ...,  1.91655465,
         1.79816862,  1.26006824]])